## Working with Data - Computer Lab for Guest Lecture Julia Lane

#### In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data.
#### OUTLINE:
#### Define a research question
#### Think about what data are available
#### Think about possible measurement errors
#### Think about the interpretation of your results
#### Inform your results by linking datasets

# 1. Define a reserach question
#### Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available¶
#### Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset.

In [489]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [490]:
# Load dataset
url = '/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9'
cols = ('Agency Name', 'Borough', 'Complaint Type', 'Community Board', 'Resolution Description')
data = pd.read_csv(url, usecols=cols)
data.head()

,Agency Name,Complaint Type,Resolution Description,Community Board,Borough
0,Department of Transportation,Street Condition,NaN,13 BROOKLYN,BROOKLYN
1,CHALL,Opinion for the Mayor,Your comments have been submitted to the Mayor...,0 Unspecified,Unspecified
2,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN
3,New York City Police Department,Illegal Parking,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN
4,HRA Benefit Card Replacement,Benefit Card Replacement,The Human Resources Administration received yo...,0 Unspecified,Unspecified


# 3. Think about possible measurement errors
#### Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc.

In [491]:
# Check if all Boroughs and Community Districts are represented in the Data
col_borough = data.Borough.unique()

In [492]:
len(col_borough)

6

In [493]:
col_borough

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [494]:
# How many unique values do we have? 
col_communityboard = data['Community Board'].unique()

In [495]:
len(data['Community Board'])

10187766

In [496]:
len(col_communityboard)

77

In [497]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.
col_communityboard

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

In [498]:
# Check for duplicates? Are these plausible?

In [499]:
data_dropduplicates = data.drop_duplicates()

In [500]:
len(data_dropduplicates['Community Board'])

70747

In [501]:
col_communityboard = data_dropduplicates['Community Board'].unique()

In [502]:
len(col_communityboard)

77

In [503]:
# What about missing values? Can you detect any patterns? 

In [504]:
data2 = data[data['Resolution Description'].isin([np.nan])==True]

In [505]:
data2.count()

Agency Name               3786751
Complaint Type            3786751
Resolution Description          0
Community Board           3786751
Borough                   3786751
dtype: int64

In [506]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?

In [507]:
data["Resolution Description"].fillna(value = data["Complaint Type"], inplace = True )

In [508]:
data3 = data[data['Resolution Description'].isin([np.nan])==True]

In [509]:
data3.count()

Agency Name               0
Complaint Type            0
Resolution Description    0
Community Board           0
Borough                   0
dtype: int64

In [510]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 

In [511]:
data['Unplausible'] = data['Community Board'].str.contains("Unspecified")

In [512]:
data[['Community Board','Unplausible']].head()

,Community Board,Unplausible
0,13 BROOKLYN,False
1,0 Unspecified,True
2,18 BROOKLYN,False
3,12 BROOKLYN,False
4,0 Unspecified,True


In [513]:
# Drop the marked districts

In [514]:
data4 = data[data['Unplausible'] == False]

In [515]:
data4.head()

,Agency Name,Complaint Type,Resolution Description,Community Board,Borough,Unplausible
0,Department of Transportation,Street Condition,Street Condition,13 BROOKLYN,BROOKLYN,False
2,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN,False
3,New York City Police Department,Illegal Parking,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN,False
5,Department of Transportation,Highway Condition,The Department of Transportation requires 10 d...,02 STATEN ISLAND,STATEN ISLAND,False
7,New York City Police Department,Noise - Commercial,Your complaint has been forwarded to the New Y...,08 MANHATTAN,MANHATTAN,False


In [516]:
len(data4['Community Board'].unique())

71

In [517]:
def check(string):
    flag = True
    if 'Unspecified' in string:
        flag = False
    elif int(string[:2]) > 20:
        flag = False
    return flag
Check = np.vectorize(check)

In [518]:
data4["Community Board"] = data4["Community Board"][Check(data4["Community Board"])]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [519]:
col_communityboard2 = data4['Community Board'].unique()

In [520]:
len(newcol_communityboard2)

59

In [521]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
# Safe reduced data frame (Community District level)

In [522]:
data5 = data4[["Community Board","Complaint Type"]]

In [523]:
rank_complaints = data5.groupby('Community Board').count()

In [524]:
rank_complaints.columns = ['Number of Complaints']

In [525]:
rank_complaints.count()

Number of Complaints    59
dtype: int64

In [526]:
rank_complaints

,Number of Complaints
Community Board,
01 BRONX,74631
01 BROOKLYN,185057
01 MANHATTAN,77974
01 QUEENS,171484
01 STATEN ISLAND,182713
02 BRONX,60257
02 BROOKLYN,121022
02 MANHATTAN,133860
02 QUEENS,114333


# 4. Think about the interpretation of your results?

#### What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data.

In [527]:
# Population by Community District
df_pop = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv")

In [529]:
# Check variables in file
df_pop.count()

FIPS                                                                                                                                                              59
cd_id                                                                                                                                                             59
Total Population                                                                                                                                                  59
Population Density (per sq. mile)                                                                                                                                 59
% Total Population: Male                                                                                                                                          59
% Total Population: 18 to 24 Years                                                                                                                                59
% Total Po

In [530]:
# How many community districts are in file? 
df_pop["cd_id"].count()

59

In [531]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
df_pop["higher_household"] = df_pop["Households: $125,000 to $149,999"] + \
                             df_pop["Households: $150,000 to $199,999"] + \
                             df_pop["Households: $200,000 or More"]

In [532]:
# Save data frame
df_pop = df_pop[["cd_id","Total Population","higher_household","Median household income (In 2014 Inflation Adjusted Dollars)"]]

In [533]:
df_pop.head()

,cd_id,Total Population,higher_household,Median household income (In 2014 Inflation Adjusted Dollars)
0,BX08,106737,6000,54224
1,BX12,134644,4761,44906
2,BX10,121209,6857,54962
3,BX11,135839,5163,47910
4,BX03,172247,1428,22041


In [534]:
# Infrastructure by Community District
df_infr = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [535]:
# Check variables in file
df_infr.count()

FIPS                                                       59
Geographic Identifier                                      59
Qualifying Name                                            59
Households                                                 59
Households: With An Internet Subscription                  59
Households: Dial-Up Alone                                  59
Households: Dsl                                            59
Households: With Mobile Broadband                          59
Households: Without Mobile Broadband                       59
Households: Cable Modem                                    59
Households: With Mobile Broadband.1                        59
Households: Without Mobile Broadband.1                     59
Households: Fiber-Optic                                    59
Households: With Mobile Broadband.2                        59
Households: Without Mobile Broadband.2                     59
Households: Satellite Internet Service                     59
Househol

In [536]:
# How many community districts are in file? 
df_infr["Qualifying Name"].count()

59

In [537]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr["household_mobile"] = df_infr["Households: With Mobile Broadband"] + \
                              df_infr["Households: With Mobile Broadband.1"] + \
                              df_infr["Households: With Mobile Broadband.2"] + \
                              df_infr["Households: With Mobile Broadband.3"] + \
                              df_infr["Households: With Mobile Broadband.4"] + \
                              df_infr["Households: Mobile Broadband Alone or With Dialup"]

In [538]:
df_infr["household_notmobile"] = df_infr["Households: Without Mobile Broadband"] + \
                                 df_infr["Households: Without Mobile Broadband.1"] + \
                                 df_infr["Households: Without Mobile Broadband.2"] + \
                                 df_infr["Households: Without Mobile Broadband.3"] + \
                                 df_infr["Households: Without Mobile Broadband.4"] + \
                                 df_infr["Households: Internet Access Without A Subscription"] + \
                                 df_infr["Households: No Internet Access"] + \
                                 df_infr["Households: Dial-Up Alone"]

In [539]:
# Aggregate internet type by high and low connections
df_infr["household_highinternet"] = df_infr["Households: Dsl"] + \
                                    df_infr["Households: Cable Modem"] + \
                                    df_infr["Households: Fiber-Optic"] + \
                                    df_infr["Households: Satellite Internet Service"] + \
                                    df_infr["Households: Two or More Fixed Broadband Types, or Other"]

In [540]:
df_infr["household_lowinternet"] = df_infr["Households: Dial-Up Alone"] + \
                                   df_infr["Households: Mobile Broadband Alone or With Dialup"] + \
                                   df_infr["Households: Internet Access Without A Subscription"]

In [435]:
# Save data frame 
df_infr = df_infr[["Qualifying Name","household_mobile","household_notmobile","household_highinternet","household_lowinternet"]]

In [541]:
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer,household_mobile,household_notmobile,household_highinternet,household_lowinternet
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,42035,35048,404,30943,3701,6987,17527,24508,29183,4731
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,44830,36700,178,31435,5087,8130,11414,33416,31137,2997
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,47050,38700,158,32333,6209,8350,12064,34986,31932,3679
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,44922,37237,122,31278,5837,7685,11749,33173,30861,3864
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,57556,42576,88,33408,9080,14980,16292,41264,34030,4785


# 5. Inform your results by linking datasets

#### Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc. https://docs.python.org/2/library/re.html

In [542]:
#reset index of rank_complaints
rank_complaints = rank_complaints.reset_index()

In [543]:
rank_complaints["Community Board"].value_counts()

15 BROOKLYN         1
04 BRONX            1
02 STATEN ISLAND    1
04 MANHATTAN        1
08 MANHATTAN        1
11 MANHATTAN        1
07 MANHATTAN        1
14 QUEENS           1
12 QUEENS           1
02 QUEENS           1
07 BROOKLYN         1
13 QUEENS           1
05 QUEENS           1
07 BRONX            1
03 BRONX            1
12 BRONX            1
01 STATEN ISLAND    1
05 BROOKLYN         1
13 BROOKLYN         1
10 BROOKLYN         1
02 BRONX            1
17 BROOKLYN         1
04 QUEENS           1
10 MANHATTAN        1
08 QUEENS           1
01 BROOKLYN         1
03 QUEENS           1
11 BROOKLYN         1
06 BRONX            1
08 BROOKLYN         1
06 MANHATTAN        1
16 BROOKLYN         1
08 BRONX            1
09 MANHATTAN        1
05 BRONX            1
09 QUEENS           1
01 BRONX            1
09 BROOKLYN         1
02 MANHATTAN        1
07 QUEENS           1
01 QUEENS           1
11 QUEENS           1
18 BROOKLYN         1
10 BRONX            1
03 BROOKLYN         1
09 BRONX  

In [544]:
# Harmonize identifier of dataframe 1
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("MA+.*", "MN")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("ST+.*", "SI")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("BROO+.*", "BK")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("QU+.*", "QN")
rank_complaints["Community Board"] = rank_complaints["Community Board"].str.replace("BRON+.*", "BX")

In [545]:
rank_complaints["cd_id"] = rank_complaints["Community Board"].str.extract('([A-Z]{2,})', expand=True)\
                         + rank_complaints["Community Board"].str.extract('([0-9]\d{0,})', expand=True)

In [546]:
rank_complaints

,Community Board,Number of Complaints,cd_id
0,01 BX,74631,BX01
1,01 BK,185057,BK01
2,01 MN,77974,MN01
3,01 QN,171484,QN01
4,01 SI,182713,SI01
5,02 BX,60257,BX02
6,02 BK,121022,BK02
7,02 MN,133860,MN02
8,02 QN,114333,QN02
9,02 SI,121132,SI02


In [547]:
# Harmonize identifier of dataframe 2

In [548]:
df_infr["Qualifying Name"].value_counts()

NYC-Bronx Community District 3 & 6--Belmont, New York                                       2
NYC-Manhattan Community District 4 & 5--Chelsea, New York                                   2
NYC-Bronx Community District 1 & 2--Hunts Point, New York                                   2
NYC-Manhattan Community District 1 & 2--Battery Park City, New York                         2
NYC-Bronx Community District 9--Castle Hill, New York                                       1
NYC-Brooklyn Community District 16--Brownsville & Ocean Hill PUMA, New York                 1
NYC-Brooklyn Community District 8--Crown Heights North & Prospect Heights PUMA, New York    1
NYC-Queens Community District 12--Jamaica, New York                                         1
NYC-Queens Community District 4--Elmhurst & South Corona PUMA, New York                     1
NYC-Brooklyn Community District 4--Bushwick PUMA, New York                                  1
NYC-Brooklyn Community District 11--Bensonhurst & Bath Beach

In [549]:
df_pop['cd_id'].unique()

array(['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05',
       'BX04', 'BX09', 'BX01', 'BX02', 'MN12', 'MN09', 'MN10', 'MN11111',
       'MN08', 'MN07', 'MN05', 'MN04', 'MN06', 'MN03', 'MN01', 'MN02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06',
       'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10',
       'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09',
       'QN12', 'QN10', 'QN14'], dtype=object)

In [550]:
#MN11111 found

In [551]:
df_pop.replace('MN11111', 'MN11', inplace=True)
df_pop['cd_id'].unique()

array(['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05',
       'BX04', 'BX09', 'BX01', 'BX02', 'MN12', 'MN09', 'MN10', 'MN11',
       'MN08', 'MN07', 'MN05', 'MN04', 'MN06', 'MN03', 'MN01', 'MN02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06',
       'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10',
       'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09',
       'QN12', 'QN10', 'QN14'], dtype=object)

In [552]:
len(df_pop["cd_id"])

59

In [553]:
# Harmonize identifier of dataframe 3

In [554]:
cdid = []
for i in df_infr["Qualifying Name"]:
    cur = i.split(' ') 
    res = [];
    if cur[0] == 'NYC-Manhattan':
        res.append("MN")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Brooklyn':
        res.append("BK")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Bronx':
        res.append("BX")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Staten':
        res.append("SI")
        num = cur[4].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    elif  cur[0] == 'NYC-Queens':
        res.append("QN")
        num = cur[3].split('--'); 
        if(int(num[0])<10):
            res.append('0'+num[0])
        else:
             res.append(num[0]) 
    cdid.append(''.join(res))
print (cdid)

['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX03', 'BX07', 'BX05', 'BX04', 'BX09', 'BX01', 'BX01', 'MN12', 'MN09', 'MN10', 'MN11', 'MN08', 'MN07', 'MN04', 'MN04', 'MN06', 'MN03', 'MN01', 'MN01', 'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07', 'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14']


In [555]:
len(cdid)

59

In [556]:
cdid = list(set(cdid))

In [557]:
len(cdid)

55

In [558]:
#after deleting duplicated ones only 55 left

In [559]:
#real cdid should be
cdid = ['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05', 'BX04', 'BX09', 'BX01', 'BX02',
       'MA12', 'MA09', 'MA10', 'MA11', 'MA08', 'MA07', 'MA04', 'MA05', 'MA06', 'MA03', 'MA01', 'MA02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06', 'BK08', 'BK16', 'BK05', 'BK18',
       'BK17', 'BK09', 'BK07', 'BK10', 'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09', 'QN12', 'QN10', 'QN14']

In [560]:
df_infr["cd_id"] = cdid

In [561]:
df_infr

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer,household_mobile,household_notmobile,household_highinternet,household_lowinternet,cd_id
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,35048,404,30943,3701,6987,17527,24508,29183,4731,BX08
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,36700,178,31435,5087,8130,11414,33416,31137,2997,BX12
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,38700,158,32333,6209,8350,12064,34986,31932,3679,BX10
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,37237,122,31278,5837,7685,11749,33173,30861,3864,BX11
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,42576,88,33408,9080,14980,16292,41264,34030,4785,BX03
5,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,42576,88,33408,9080,14980,16292,41264,34030,4785,BX06
6,3603706,79500US3603706,"NYC-Bronx Community District 7--Bedford Park, ...",47252,31468,0,1598,502,1096,18280,...,38899,0,31021,7878,8353,16709,30543,29844,6279,BX07
7,3603707,79500US3603707,NYC-Bronx Community District 5--Morris Heights...,44699,26332,54,956,352,604,11333,...,34397,54,25772,8571,10302,15857,28842,25216,4096,BX05
8,3603708,79500US3603708,"NYC-Bronx Community District 4--Concourse, New...",47935,29376,316,301,73,228,14610,...,34692,316,27971,6405,13243,18342,29593,26686,6391,BX04
9,3603709,79500US3603709,"NYC-Bronx Community District 9--Castle Hill, N...",64011,45976,180,1536,296,1240,23999,...,53072,180,45261,7631,10939,12892,51119,44133,3714,BX09


In [562]:
# Link the 3 dataframes

In [563]:
combine = pd.merge(df_infr, rank_complaints, on = "cd_id",how ="inner")

In [564]:
combine = pd.merge(combine, df_pop, on = "cd_id" )

In [565]:
combine

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,household_mobile,household_notmobile,household_highinternet,household_lowinternet,cd_id,Community Board,Number of Complaints,Total Population,higher_household,Median household income (In 2014 Inflation Adjusted Dollars)
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,17527,24508,29183,4731,BX08,08 BX,92733,106737,6000,54224
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,11414,33416,31137,2997,BX12,12 BX,168455,134644,4761,44906
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,12064,34986,31932,3679,BX10,10 BX,95496,121209,6857,54962
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,11749,33173,30861,3864,BX11,11 BX,122081,135839,5163,47910
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,16292,41264,34030,4785,BX03,03 BX,75134,172247,1428,22041
5,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,16292,41264,34030,4785,BX06,06 BX,99547,172247,1428,22041
6,3603706,79500US3603706,"NYC-Bronx Community District 7--Bedford Park, ...",47252,31468,0,1598,502,1096,18280,...,16709,30543,29844,6279,BX07,07 BX,185729,135893,1412,35495
7,3603707,79500US3603707,NYC-Bronx Community District 5--Morris Heights...,44699,26332,54,956,352,604,11333,...,15857,28842,25216,4096,BX05,05 BX,161078,132850,854,20872
8,3603708,79500US3603708,"NYC-Bronx Community District 4--Concourse, New...",47935,29376,316,301,73,228,14610,...,18342,29593,26686,6391,BX04,04 BX,181953,141467,1375,27203
9,3603709,79500US3603709,"NYC-Bronx Community District 9--Castle Hill, N...",64011,45976,180,1536,296,1240,23999,...,12892,51119,44133,3714,BX09,09 BX,147457,190126,4427,36058


In [566]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?

#### Yes, I think so. For example,compare BX08 and BX12:BX12 has more population but less household mobiles, then it has more number of complaints than that of BX08.